# Working with GPUs in OpenVINO™

This tutorial provides a high-level overview of working with Intel GPUs in OpenVINO. It shows users how to use Query Device to list system GPUs and check their properties, and it explains some of the key properties. It shows how to compile a model on GPU with performance hints and how to use multiple GPUs using MULTI or CUMULATIVE_THROUGHPUT. 

The tutorial shows example commands for benchmark_app that users can run to compare GPU performance in different configurations. It also provides code for a basic end-to-end application that compiles a model on GPU and uses it to run inference.

## Introduction

1. Background and context on how GPUs are used to speed up inference
2. Introduce OpenVINO’s ability to run inference with GPUs
3. How to configure OpenVINO to work with GPUs (link to Configuration for GPU with OpenVINO page)

## Checking GPUs with Query Device

1. List GPUs with ie.get_available_devices
2. Check properties with ie.get_property
3. Brief descriptions of key properties

In this section we will see how to list the available GPUs and check their properties. Some of the key properties will also be defined.

### List GPUs with core.get_available_devices


Firstly, in order to use GPUs, we must make sure our system is detecting them correctly.
Running the following cell should output a list of compatible OpenVINO devices, in which our Intel GPUs should appear.

In [ ]:
from openvino.runtime import Core

core = Core()
core.available_devices

If the GPUs are installed correctly in the system and still don't appear in the list, we should follow the steps described [here](https://docs.openvino.ai/latest/openvino_docs_install_guides_configurations_for_intel_gpu.html) and try again. Once we have the GPUs working with OpenVINO we can proceed with the next sections.

### Check properties with core.get_property

Now, to get information and customize the behavior of our GPUs, we can use device properties. Devices in OpenVINO, such as CPUs and GPUs, have two types of properties: read-only and read-write. The former mainly shows information about the hardware itself like the device name or supported data types, while the latter allows us to tweak how the model is compiled, for instance to reduce latency or increase throughput.

So, to get the value of a property, such as the device name, we can use the `core.get_property` method as follows

In [ ]:
core.get_property("CPU", "FULL_DEVICE_NAME")

Each device also has a specific property, called `SUPPORTED_PROPERTIES`, that allows us to see all the available properties in the device (including the `SUPPORTED_PROPERTIES` itself). To do this, we repeat the above command

In [ ]:
core.get_property("CPU", "SUPPORTED_PROPERTIES")

Note that the value for each property has either a "RO" or "RW", which corresponds to the two types mentioned previously, "**R**ead-**O**nly" and "**R**-**W**rite" respectively.

### Brief descriptions of key properties

Each device has several properties as seen in the last command. Some of the key properties would be

* Read-Only
    * `FULL_DEVICE_NAME` - The product name of the GPU and whether it is an integrated or discrete GPU (iGPU or dGPU).
    * `OPTIMIZATION_CAPABILITIES` - The model data types (INT8, FP16, FP32, etc) that are supported by this GPU.
    * `GPU_EXECUTION_UNITS_COUNT` - The execution cores available in the GPU's architecture, which is a relative measure of the GPU's processing power.
    * `RANGE_FOR_STREAMS` - The number of processing streams available on the GPU that can be used to execute parallel inference requests. When compiling a model in LATENCY or THROUGHPUT mode, OpenVINO will automatically select the best number of streams for low latency or high throughput.
    * `DEVICE_GOPS` - The Giga operations per second count (GFLOPS) for each precision the device supports.
* Read-Write
    * `PERFORMANCE_HINT` - A high-level way to tune the device for a specific performance metric, such as latency or throughput, without worrying about device-specific settings.
    * `INFERENCE_PRECISION_HINT` - A high-level way to specify which model data type to use for inference.

To sum up this section, we can check the value for each property by simply looping through the dictionary returned by `core.get_property("GPU", "SUPPORTED_PROPERTIES")` and then querying for that property.

In [ ]:
supported_properties = core.get_property("CPU", "SUPPORTED_PROPERTIES")
indent = len(max(supported_properties, key=len))
for prop in supported_properties:
    print(f"{prop:>{indent}}:", core.get_property("CPU", prop))

## Compiling a Model on GPU

1. Compile with default configuration (ie.compile_model(model, “GPU”)
2. Throughput and latency performance hints
3. Using multiple GPUs with multi-device and cumulative throughput

We now know what is a GPU, how to check if we have one and its properties but, how do we actually **use** one?

### Compile with default configuration (ie.compile_model(model, “GPU”)

In fact, due to the [AUTO plugin](https://docs.openvino.ai/latest/openvino_docs_OV_UG_supported_plugins_AUTO.html#how-auto-works), we might already be using GPUs if they are properly recognized by OpenVINO. Despite this, if we want to use a specific device, we can do so by compiling our models as follows

In [ ]:
# load any model
model = core.read_model(model="../001-hello-world/model/v3-small_224_1.0_float.xml")

# compile the loaded model using the GPU device
compiled_model = core.compile_model(model, "CPU")

Note that above we are using `"GPU"` which is an alias for `"GPU.0"` according to the [docs](https://docs.openvino.ai/latest/openvino_docs_OV_UG_supported_plugins_GPU.html). Actually, as expected, any of the values returned by `core.available_devices` are valid device specifiers.

Once we have a compiled model, we can check for its properties, just as we did with the GPU devices in the previous section. Instead of calling `core.get_property` with a specific device, we use `compiled_model.get_property` directly with the property name.

In [ ]:
compiled_model.get_property("SUPPORTED_PROPERTIES")

As we can appreciate from the above result, the `PERFORMANCE_HINT` property defined earlier reappeared here, let's take a deeper look at it.

### Throughput and latency performance hints

Essentially, the `PERFORMANCE_HINT` allows us to easily tweak either our device or model properties to better suit certain tasks. Currently, it supports three values: `"LATENCY"` which prioritizes short response times for each inference, `"THROUGHPUT"` which helps inferring large amounts of data at the same time like a video feed, and `"CUMULATIVE_THROUGHPUT"` which will see later. The hint's behavior is the same in both the device and in the model, except for the fact that in the device the hint can be considered a "global" property, i.e all models created on that device will use the same value of the hint, whereas in the compiled model acts as a "local" one, i.e only affects that model. See [the docs](https://docs.openvino.ai/latest/openvino_docs_OV_UG_query_api.html#setting-properties-globally) for more info.

Until now we have only queried either the device or the compiled model for properties so, how can we modify those that are read-write such as the `PERFOMANCE_HINT`?

To modify properties in the device, we can use the `core.set_property` method, in which instead of just using the property name, a dictionary is required specifying the value as well. For example, if we want to change the `PERFORMANCE_HINT` to improve latency we can do it like

In [ ]:
core.set_property("CPU", {"PERFORMANCE_HINT": "LATENCY"})
core.get_property("CPU", "PERFORMANCE_HINT")

If instead we only want to affect a certain model, we need to add the dictionary as the third argument in the `core.compile_model` method. For instance, to improve throughtput for just this model the following works

In [ ]:
compiled_model = core.compile_model(model, "CPU", {"PERFORMANCE_HINT": "THROUGHPUT"})
compiled_model.get_property("PERFORMANCE_HINT")

### Using multiple GPUs with multi-device and cumulative throughput

The latency and throughput hints mentioned above are great and can make a difference when used adequately but they usually use just one device, either due to the [AUTO plugin](https://docs.openvino.ai/latest/openvino_docs_OV_UG_supported_plugins_AUTO.html#how-auto-works) or by manual specification of the device name as we did above. In case we have multiple devices, such as an integrated and discrete GPU, we could use both at the same time to improve the utlization of our resources. In order to do this, OpenVINO provides a virtual device called [MULTI](https://docs.openvino.ai/nightly/openvino_docs_OV_UG_Running_on_multiple_devices.html), which is just a combination of our existent devices that knows how to split inference work between them, leveraging the capabilities of each device.

So, as an example, if we want to use both our integrated and discrete GPUs and the CPU at the same time, we can compile our model as follows:

In [ ]:
compiled_model = core.compile_model(model=model, device_name="MULTI:GPU.1,GPU.0,CPU")

Note that we always need to explicitly specify the device list for MULTI to work, as otherwise MULTI does not know which devices are available for inference. However, this is not the only way to use multiple devices in OpenVINO. There is another `PERFORMANCE_HINT` called CUMULATIVE_THROUGHPUT that works similar to MULTI, except it uses the devices automatically selected by AUTO. This way, we don't need to manually specify which devices to use. Here's an example showing how to use CUMULATIVE_THROUGHPUT, equivalent to the MULTI one:


In [ ]:
compiled_model = core.compile_model(model, "AUTO", {"PERFORMANCE_HINT": "CUMULATIVE_THROUGHPUT"})

## Performance Comparison with benchmark_app

1. Commands showing users how to run benchmark_app on GPU with various performance hints
2. Show performance results with a basic model (person-detection-0303, perhaps)

Given all the different options available when compiling a model, it may be difficult to know which settings work best for a certain application. Thankfully, OpenVINO provides a performance benchmarking tool called `benchmark_app`.

### Commands showing users how to run benchmark_app on GPU with various performance hints

The basic syntax of benchmark_app is as follows:

```bash
benchmark_app -m PATH_TO_MODEL -d TARGET_DEVICE -hint {throughput,cumulative_throughput,latency,none}
```
where TARGET_DEVICE is any device shown by `core.available_devices` as well as the MULTI and AUTO devices we saw previously, and the value of hint should be one of the values between brackets. 

Note that benchmark_app only requires the model path to run but both the device and hint arguments will be useful to us. For more advanced usages, the tool itself has other options that can be checked by running `benchmark_app -h` or reading the [docs](https://docs.openvino.ai/latest/openvino_inference_engine_tools_benchmark_tool_README.html).

So, as an example, if we want to benchmark a simple model using a GPU with a latency focus, we would run the following:

In [17]:
!benchmark_app -m model/v3-small_224_1.0_float.xml -d CPU -hint latency

[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2022.3.0-9052-9752fafe8eb-releases/2022/3
[ INFO ] 
[ INFO ] Device info:
[ INFO ] CPU
[ INFO ] Build ................................. 2022.3.0-9052-9752fafe8eb-releases/2022/3
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 21.41 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     input:0 (node: input) : f32 / [...] / [1,224,224,3]
[ INFO ] Model outputs:
[ INFO ]     MobilenetV3/Predictions/Softmax:0 (node: MobilenetV3/Predictions/Softmax) : f32 / [...] / [1,1001]
[Step 5/11] Resizing model to match image sizes and given batch
[ INFO ] Model batch size: 1
[Step 6/11] Configuring input of the model
[ INFO ] Model inputs:
[ INFO ]     input:0 (node: input) : u8 / [N,H,W,C] /

### Show performance results with a basic model (person-detection-0303, perhaps)

In [ ]:
import pandas as pd
pd.read_csv("benchmark_report.csv", sep=";")

## Basic Application Using GPUs

1. Provide end-to-end sample code for running inference on GPU in a basic application

We will now show an end-to-end object detection example running on GPU.

    1. Import necessary packages
    2. Download and convert ssdlite-mobilenet-v2 model
    3. Read ssd-mobilenet model and compile model on GPU in THROUGHPUT mode
    4. Load every frame of a video and resize it to shape expected by model
    5. Set up AsyncInferQueue, process every frame of video and store results, time how long it takes
    6. Post-process results to print every object detected

### Import Necessary Packages

In [101]:
import os
import sys
import time
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Markdown, Video, display
from openvino.runtime import AsyncInferQueue, CompiledModel, Core, InferRequest

core = Core()
core.available_devices

['CPU']

### Download the Model

In [48]:
# A directory where the model will be downloaded.
base_model_dir = "model"

# The name of the model from Open Model Zoo
model_name = "ssdlite_mobilenet_v2"

model_path = f"model/public/{model_name}"

if not os.path.exists(model_path):
    download_command = f"omz_downloader " \
                       f"--name {model_name} " \
                       f"--output_dir {base_model_dir} " \
                       f"--cache_dir {base_model_dir}"
    ! $download_command

### Convert the Model to OpenVINO IR format

In [49]:
precision = "FP16"

# The output path for the conversion.
converted_model_path = f"model/public/{model_name}/{precision}/{model_name}.xml"

if not os.path.exists(converted_model_path):
    convert_command = f"omz_converter " \
                      f"--name {model_name} " \
                      f"--download_dir {base_model_dir} " \
                      f"--precisions {precision}"
    ! $convert_command

### Compile the Model

In [50]:
model = core.read_model(model=converted_model_path)
compiled_model = core.compile_model(model=model, device_name="CPU", config={"PERFORMANCE_HINT": "THROUGHPUT"})

# Get the input and output nodes.
input_layer = compiled_model.input(0)
output_layer = compiled_model.output(0)

input_keys = list(compiled_model.inputs)
print(input_keys)

# Get the input size.
num, height, width, channels = input_layer.shape
print('Model input shape:', num, height, width, channels)

[<ConstOutput: names[image_tensor, image_tensor:0] shape[1,300,300,3] type: u8>]
Model input shape: 1 300 300 3


### Define Model Output Classes

In [51]:
# COCO classes! (the dataset, not the dog)
classes = [
    "background", "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train",
    "truck", "boat", "traffic light", "fire hydrant", "street sign", "stop sign",
    "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow", "elephant",
    "bear", "zebra", "giraffe", "hat", "backpack", "umbrella", "shoe", "eye glasses",
    "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite",
    "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle",
    "plate", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair",
    "couch", "potted plant", "bed", "mirror", "dining table", "window", "desk", "toilet",
    "door", "tv", "laptop", "mouse", "remote", "keyboard", "cell phone", "microwave", "oven",
    "toaster", "sink", "refrigerator", "blender", "book", "clock", "vase", "scissors",
    "teddy bear", "hair drier", "toothbrush", "hair brush"
]

### Load and Preprocess Video Frames 

In [168]:
### Load video
video_file = "../data/video/Coco Walking in Berkeley.mp4"
video = cv2.VideoCapture(video_file)
framebuf = []

print('Loading video...')
while video.isOpened():
    ret, frame = video.read()
    if not ret:
        print('Video loaded!')
        video.release()
        break
    
    # Preprocess frames - convert them to shape expected by model
    input_frame = cv2.resize(src=frame, dsize=(width, height), interpolation=cv2.INTER_AREA)
    input_frame = np.expand_dims(input_frame, axis=0)

    # Append frame to framebuffer
    framebuf.append(input_frame)
    

print('Frame shape: ', framebuf[0].shape)
print('Number of frames: ', len(framebuf))

# Show original video file
Video(video_file)

Loading video...
Video loaded!
Frame shape:  (1, 300, 300, 3)
Number of frames:  288


### Set up Asynchronous Pipeline

#### Callback Definition

In [56]:
def completion_callback(infer_request: InferRequest, results) -> None:
    predictions = next(iter(infer_request.results.values()))
    results.append(predictions[:10]) # Grab first 10 predictions for this frame

#### Create Async Pipeline

In [57]:
# Create async queue with optimal number of infer requests
infer_queue = AsyncInferQueue(compiled_model)
infer_queue.set_callback(completion_callback)

### Perform Inference

In [108]:
results = []
start_time = time.time()
for i, input_frame in enumerate(framebuf):
    infer_queue.start_async({0: input_frame}, results)

infer_queue.wait_all()
stop_time = time.time()

total_time = stop_time - start_time
time_per_frame = total_time / len(framebuf)
fps = len(framebuf) / total_time
print(f'Total time to inference all frames: {total_time:.3f}s')
print(f'Time per frame: {time_per_frame:.6f}s ({fps:.3f} FPS)')

Total time to inference all frames: 1.863s
Time per frame: 0.006467s (154.623 FPS)


### Process Results

In [167]:
# Set minimum detection threshold
min_thresh = .6

# Load video
video = cv2.VideoCapture(video_file)

# Get video parameters
frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video.get(cv2.CAP_PROP_FPS))
fourcc = int(video.get(cv2.CAP_PROP_FOURCC))

output = cv2.VideoWriter('output.mp4', fourcc, fps, (frame_width, frame_height))

while video.isOpened():
    current_frame = int(video.get(cv2.CAP_PROP_POS_FRAMES))
    ret, frame = video.read()
    if not ret:
        print('Video loaded!')
        output.release()
        video.release()
        break

    # prediction contains [image_id, label, conf, x_min, y_min, x_max, y_max] according to model
    for prediction in np.squeeze(results[current_frame]):
        if prediction[2] > min_thresh:
            x_min = int(prediction[3] * frame_width)
            y_min = int(prediction[4] * frame_height)
            x_max = int(prediction[5] * frame_width)
            y_max = int(prediction[6] * frame_height)
            label = classes[int(prediction[1])]
            
            # Draw a bounding box with its label above it
            image = cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0,255,0), 2)
            cv2.putText(image, label, (x_min, y_min - 10), cv2.FONT_ITALIC, 1, (255,0,0), 2)

    output.write(frame)

# Show output
Video("output.mp4")

Video loaded!


## Conclusion

1. GPUs are easy to use with OpenVINO and considerably boost performance
2. Links to OpenVINO documentation where readers can learn more